## 1&nbsp;&nbsp;&nbsp;Extraire du texte brut à partir de la balise `<body>`

In [1]:
import os
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup

test_path = './test/'
txt_path = './txt_non_corr/'
 

    
with open(txt_path+'txt_non_corr_3.txt', 'w') as txt:


    for filename in os.listdir(test_path)[:2]:
        f = os.path.join(test_path,filename)

        with open(f, 'r') as f:


            # 'xml' is the parser used. For html files, which BeautifulSoup is typically used for, it would be 'html.parser'.
            soup = BeautifulSoup(f, 'xml')
            names = soup.find('body')
            n = names.text
            print(n)
            txt.write(n)











Poesies
DE


Valentin
(HENRI BOURETTE
PUBLIEES PAR

MM. T. LAROUSSILHE ET J. COMBARIEU


A. LEMERRE

LIBRAIRE-EDITEUR
7-31. Passage Choiseul
PARIS

J. GIRMA

LIBRAIRE
24. Boulevard Gambetta
GAHORS
M DCCC LXXXV




Poésies de valentin

JUSTIFICATION DU TIRAGE
Papier vélin teinté. 460 exemplaires
Papier de Hollande (numérotés). 40 —
500 exemplaires


cliché Honore fils CAHORS



Poesies
DE


Valentin
(HENRI BOURETTE) 
PUBLIÉES PAR

F. LAROUSSlLHE ET J. COMBARIEU


A. LEMERRE

I IBRAIRE-EDITEUR
27-37, Passage Choiseul
PARIS

J. GIRMA

LIBRAIRE
24, Boulevard Gambetta
CAHORS
M DCCC LXXXV




Notice biographique
CAHORS n’a pas donné le jour à Valentin (Henri


Bourette). Notre poète était né dans le nord


de la France où son père occupait l’emploi


de vérificateur des Contributions directes. Mais il fut


amené tout enfant dans l’ancienne capitale du Quercy,


evenue ensuite la résidence de ses parents et de son


seul frère, âgé de vingt ans de plus que lui. Il fit ses


classes

## 1a.&nbsp;&nbsp;&nbsp;Retirer les lignes vides après l’extraction du texte

In [2]:
!ex -s +'v/\S/d' -cwq './txt_non_corr/txt_non_corr_3.txt'

## 2&nbsp;&nbsp;&nbsp;Lancer [`spacy`](https://spacy.io/models/fr) sur le fichier `.txt`

In [3]:
import spacy, csv, os

path = './txt_non_corr/txt_non_corr_3.txt'

path_of_the_directory_2= './csv_non_corr/' 
 
nlp = spacy.load("fr_core_news_lg") # faire pareil pour fr_core_news_md et fr_core_news_lg
nlp.max_length = 1511212

with open(path, encoding='utf-8', errors='ignore') as file_in:
    with open(path_of_the_directory_2+'EN_non_corr_lg_3.csv', 'w') as doc_csv:

    # create the csv writer
        writer = csv.writer(doc_csv)

        text = file_in.read()
        doc = nlp(text)
        for ent in doc.ents:
            #print(ent, ent.label_)
            writer.writerow([str(ent)])   
            
 


## Évaluation avec [`Difflib`](https://docs.python.org/3/library/difflib.html)

Démo

In [21]:
from difflib import SequenceMatcher
line1 = "aaa bbb"
line2 = "aaa abb"
sm = SequenceMatcher(a=line1, b=line2)
print (sm.ratio())

0.8571428571428571


## Sortie HTML (visualisation des diffs)

In [45]:
from difflib import HtmlDiff

line1 = "./csv_non_corr/EN_non_corr_lg_3.csv"
line2 = "./csv_corr/EN_corr_lg_3.csv"
d = HtmlDiff()
difference = d.make_file(open(line1).readlines(), open(line2).readlines())
with open("diff.html", "w") as f:
    for line in difference.splitlines():
        print (line, file=f)
        
from IPython.display import IFrame

IFrame(src='./diff.html', width=700, height=600)


In [64]:
import difflib

file1 = "./csv_non_corr/EN_non_corr_lg_3.csv"
file2 = "./csv_corr/EN_corr_lg_3.csv"

diff = difflib.ndiff(open(file1).readlines(),open(file2).readlines())
print (''.join(diff))

+ BOULETTE
+ COMBATTRE
- "Poesies
- DE
- Valentin
- "
- HENRI
- PUBLIEES
- MM. T. LAROUSSILHE
- "Passage Choiseul
- PARIS
- J. GIRMA
- LIBRAIRE
- 24"
- "Boulevard Gambetta
- GAHORS
- M"
- "DCCC LXXXV
- "
- Papier vélin teinté
- Papier de Hollande
- "Honore fils CAHORS
- Poesies
- DE
- Valentin"
- HENRI
- PUBLIÉES
- "Passage Choiseul
- PARIS
- J. GIRMA
- "
- "Boulevard Gambetta
  CAHORS
+ XXX
+ BOULETTE)
+ COMBATTRE
+ XXX
- M"
- DCCC
- CAHORS
- Valentin
- "Henri
- Bourette"
?    ^    ^
+ Boulette).
?    ^    ^^
+ classique.
+ avertissements
+ gris,
+ gagnerait
- la France
- Contributions
- Quercy
- Valette
- Lycée de
- Cahors
- "NOTICE BIOGRAPHIQUE
- franchir les limites."
- Réformateur du Lot
- "Molière,
- Racine"
- Alfred de Musset
- Lamartine
- "Barbier,
- Shakespeare"
- Milton
- Dante
- "Le Tasse, son
- père"
- France
- l’Afrique
- Valentin
- NOTICE BIOGRAPHIQUE III
- Valentin
- Valentin
- Algérie
- "IV NOTICE BIOGRAPHIQUE
- des inoffensives"
- Muse
- Afrique
- Silo
- A
- Marseille


### Aperçu des différences entre les EN avant et après la correction d'OCR

In [57]:
import difflib

ratio = "./ratio.txt"

file_non_corr_3 = "./csv_non_corr/EN_non_corr_lg_3.csv"
file_non_corr_6 = "./csv_non_corr/EN_non_corr_lg_6.csv"
file_non_corr_9 = "./csv_non_corr/EN_non_corr_lg_9.csv"

file_corr_3 = "./csv_corr/EN_corr_lg_3.csv"
file_corr_6 = "./csv_corr/EN_corr_lg_6.csv"
file_corr_9 = "./csv_corr/EN_corr_lg_9.csv"

diff = difflib.ndiff(open(file_non_corr_3).readlines(),open(file_corr_3).readlines())
diff_out = ''.join(diff)
print (diff_out)

with open(ratio, 'w') as r:
    r.write(diff_out)

+ BOULETTE
+ COMBATTRE
- "Poesies
- DE
- Valentin
- "
- HENRI
- PUBLIEES
- MM. T. LAROUSSILHE
- "Passage Choiseul
- PARIS
- J. GIRMA
- LIBRAIRE
- 24"
- "Boulevard Gambetta
- GAHORS
- M"
- "DCCC LXXXV
- "
- Papier vélin teinté
- Papier de Hollande
- "Honore fils CAHORS
- Poesies
- DE
- Valentin"
- HENRI
- PUBLIÉES
- "Passage Choiseul
- PARIS
- J. GIRMA
- "
- "Boulevard Gambetta
  CAHORS
+ XXX
+ BOULETTE)
+ COMBATTRE
+ XXX
- M"
- DCCC
- CAHORS
- Valentin
- "Henri
- Bourette"
?    ^    ^
+ Boulette).
?    ^    ^^
+ classique.
+ avertissements
+ gris,
+ gagnerait
- la France
- Contributions
- Quercy
- Valette
- Lycée de
- Cahors
- "NOTICE BIOGRAPHIQUE
- franchir les limites."
- Réformateur du Lot
- "Molière,
- Racine"
- Alfred de Musset
- Lamartine
- "Barbier,
- Shakespeare"
- Milton
- Dante
- "Le Tasse, son
- père"
- France
- l’Afrique
- Valentin
- NOTICE BIOGRAPHIQUE III
- Valentin
- Valentin
- Algérie
- "IV NOTICE BIOGRAPHIQUE
- des inoffensives"
- Muse
- Afrique
- Silo
- A
- Marseille


## Ratio entre les EN issues de différentes portions de texte

Texte non corrigé <br>

`EN_non_corr_lg_3.csv` vs. `EN_non_corr_lg_6.csv`

In [42]:
sm = SequenceMatcher(a=file_non_corr_3, b=file_non_corr_6)
print (sm.ratio())

0.9714285714285714


Texte corrigé <br>

`EN_corr_lg_3.csv` vs. `EN_corr_lg_6.csv`

In [43]:
sm = SequenceMatcher(a=file_corr_3, b=file_corr_6)
print (sm.ratio())

0.9629629629629629


#### Pourquoi le même ratio ?

In [69]:
import difflib

file_non_corr_3 = "./csv_non_corr/EN_non_corr_lg_3.csv"
file_non_corr_6 = "./csv_non_corr/EN_non_corr_lg_6.csv"
file_non_corr_9 = "./csv_non_corr/EN_non_corr_lg_9.csv"

file_corr_3 = "./csv_corr/EN_corr_lg_3.csv"
file_corr_6 = "./csv_corr/EN_corr_lg_6.csv"
file_corr_9 = "./csv_corr/EN_corr_lg_9.csv"

diff = difflib.ndiff(open(file_non_corr_3).readlines(),open(file_non_corr_9).readlines())
#diff_out = ''.join(diff)
#print (diff_out)

sm = SequenceMatcher(open(file_non_corr_3).readlines(), open(file_non_corr_9).readlines())
print (sm.ratio())

0.0


## Ratio entre les EN avant et après la correction d'OCR

In [44]:
sm = SequenceMatcher(a=file_non_corr_3, b=file_corr_3)
print (sm.ratio())

0.8709677419354839


In [5]:
import difflib as dl
file_non_corr_3 = "./csv_non_corr/EN_non_corr_lg_3.csv"
file_corr_3 = "./csv_corr/EN_corr_lg_3.csv"
dl.context_diff(s1, s2)

NameError: name 's1' is not defined

In [24]:
import difflib as dl

# opening the file in read mode
my_file = open("./csv_non_corr/EN_non_corr_lg_3.csv", "r")
my_file_2 = open("./csv_corr/EN_corr_lg_3.csv", "r")

# reading the file
data = my_file.read()
data_2 = my_file_2.read()

# replacing end of line('/n') with ' ' and
# splitting the text it further when '.' is seen.
data_into_list = data.replace('\n', ' ').split(".")
data_into_list_2 = data_2.replace('\n', ' ').split(".")

# printing the data
# print(data_into_list)



for diff in dl.unified_diff(data_into_list, data_into_list_2):
    print(diff)
    
my_file.close()
my_file_2.close()


--- 

+++ 

@@ -1,99 +1,157 @@

-"Poesies DE Valentin " HENRI PUBLIEES MM
- T
- LAROUSSILHE "Passage Choiseul PARIS J
- GIRMA LIBRAIRE 24" "Boulevard Gambetta GAHORS M" "DCCC LXXXV " Papier vélin teinté Papier de Hollande "Honore fils CAHORS Poesies DE Valentin" HENRI PUBLIÉES "Passage Choiseul PARIS J
- GIRMA " "Boulevard Gambetta CAHORS M" DCCC CAHORS Valentin "Henri Bourette" la France Contributions Quercy Valette Lycée de Cahors "NOTICE BIOGRAPHIQUE franchir les limites
-" Réformateur du Lot "Molière, Racine" Alfred de Musset Lamartine "Barbier, Shakespeare" Milton Dante "Le Tasse, son père" France l’Afrique Valentin NOTICE BIOGRAPHIQUE III Valentin Valentin Algérie "IV NOTICE BIOGRAPHIQUE des inoffensives" Muse Afrique Silo A Marseille de France "Valentin avait" "NOTICE BIOGRAPHIQUE V " Afrique Afrique "Cour d’appel de Saint-Denis" Réunion "Cahors
- Valentin" Bordeaux Cahors "VI NOTICE BIOGRAPHIQUE " M
- H
- Valette M
- Delpérier Réformateur du Lot Mme Guilhou M
- Escoffier Financ

In [39]:
seq_matcher = dl.SequenceMatcher(None, data_into_list, data_into_list_2)
for tag, i1, i2, j1, j2 in seq_matcher.get_opcodes():
    print(f'{tag:7} data_into_list[{i1}:{i2}] --> data_into_list_2[{j1}:{j2}] {data_into_list[i1:i2]!r:>6} --> {data_into_list_2[j1:j2]!r}') 

replace data_into_list[0:99] --> data_into_list_2[0:157] ['"Poesies DE Valentin " HENRI PUBLIEES MM', ' T', ' LAROUSSILHE "Passage Choiseul PARIS J', ' GIRMA LIBRAIRE 24" "Boulevard Gambetta GAHORS M" "DCCC LXXXV " Papier vélin teinté Papier de Hollande "Honore fils CAHORS Poesies DE Valentin" HENRI PUBLIÉES "Passage Choiseul PARIS J', ' GIRMA " "Boulevard Gambetta CAHORS M" DCCC CAHORS Valentin "Henri Bourette" la France Contributions Quercy Valette Lycée de Cahors "NOTICE BIOGRAPHIQUE franchir les limites', '" Réformateur du Lot "Molière, Racine" Alfred de Musset Lamartine "Barbier, Shakespeare" Milton Dante "Le Tasse, son père" France l’Afrique Valentin NOTICE BIOGRAPHIQUE III Valentin Valentin Algérie "IV NOTICE BIOGRAPHIQUE des inoffensives" Muse Afrique Silo A Marseille de France "Valentin avait" "NOTICE BIOGRAPHIQUE V " Afrique Afrique "Cour d’appel de Saint-Denis" Réunion "Cahors', ' Valentin" Bordeaux Cahors "VI NOTICE BIOGRAPHIQUE " M', ' H', ' Valette M', ' Delpérier Réforma

In [42]:
for a,b in data_into_list:     
    print('{} => {}'.format(a,b))  
    for i,s in enumerate(difflib.ndiff(data_into_list,data_into_list_2)):
        if s[0]==' ': continue
        elif s[0]=='-':
            print(u'Delete "{}" from position {}'.format(s[-1],i))
        elif s[0]=='+':
            print(u'Add "{}" to position {}'.format(s[-1],i))    
    print()      

ValueError: too many values to unpack (expected 2)